In [1]:
!nvidia-smi

Mon Apr 18 02:44:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 7.4 MB/s 
     |████████████████████████████████| 77 kB 6.9 MB/s 
     |████████████████████████████████| 6.6 MB 32.8 MB/s 
     |████████████████████████████████| 596 kB 60.9 MB/s 
     |████████████████████████████████| 895 kB 51.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Read the following link to get kaggle.json https://www.kaggle.com/general/74235


In [2]:
%%shell
mkdir ~/.kaggle
cp kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c fake-news

 71% 33.0M/46.5M [00:00<00:00, 74.4MB/s]
100% 46.5M/46.5M [00:00<00:00, 88.5MB/s]


In [4]:
!unzip /content/fake-news.zip

Archive:  /content/fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [7]:
from tqdm.auto import tqdm
import pandas as pd
import tensorflow as tf
import numpy as np
import string
import re
from os.path import join
import os
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from transformers import create_optimizer
from sklearn.metrics import classification_report,confusion_matrix
from transformers import AutoConfig
from itertools import chain
from collections import Counter

In [8]:
fake_news_folder="fake_news"
fake_new_dataset="raw_dataset"
fake_new_models="models"
name="bert-base-uncased"

In [9]:
root_path="/content/drive/MyDrive"
fake_news_folder_path=join(root_path,fake_news_folder)
fake_news_dataset_path=join(root_path,fake_news_folder,fake_new_dataset)
fake_news_models_path=join(root_path,fake_news_folder,fake_new_models)
model_path=f'{fake_news_models_path}/{name}/'

In [10]:
os.makedirs(model_path, exist_ok=True)
os.makedirs(fake_news_dataset_path, exist_ok=True)

# Load fake news dataset from kaggle

In [13]:
df=pd.read_csv("/content/train.csv",index_col=False)
df=df.drop(['id','title','author'],axis=1)
df=df.sample(frac=1)
df

,text,label
2447,BERLIN — Austria’s highest court threw out ...,0
6975,British ex-soldier cuts off own toes with plie...,1
3083,These strange sounds were recorded in Slovakia...,1
15732,Pin 4 \n( ANTIMEDIA ) Democratic presidential ...,1
15751,In a leaked email between the Democratic Candi...,1
...,...,...
17489,The president is expected to formally announce...,0
15233,Historian John Heiser from the Gettysburg Nati...,0
5651,Tesla fired a female engineer after claiming t...,0
11545,"Joe McKnight, who played three seasons for the...",0


Split raw dataset to train set and validation set

In [14]:
train=df.iloc[:-2600]
valid=df.iloc[-2600:]

In [15]:
len(train),len(valid)

(18200, 2600)

Check data whether it is balanced or not

In [16]:
for i in range(2):
  samples=train[train['label']==i]
  print(f'label: {i}\thas {len(samples)} samples')

label: 0	has 9128 samples
label: 1	has 9072 samples


In [17]:
for i in range(2):
  samples=valid[valid['label']==i]
  print(f'label: {i}\thas {len(samples)} samples')

label: 0	has 1259 samples
label: 1	has 1341 samples


In [18]:
train.to_csv(join(fake_news_dataset_path,"train.csv"),index=False)

In [19]:
valid.to_csv(join(fake_news_dataset_path,"valid.csv"),index=False)

# Clean and preprocessing

##Basic clean data

In [31]:
train=pd.read_csv(join(fake_news_dataset_path,"train.csv"),index_col=False)
train=train.dropna(subset=['label', 'text'])

valid=pd.read_csv(join(fake_news_dataset_path,"valid.csv"),index_col=False)
valid=valid.dropna(subset=['label', 'text'])


In [32]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [33]:
# """This function is from The Stanford Question Answering Dataset evaluate"""
# def normalize(s):
  
#   """Lower text and remove punctuation, articles and extra whitespace."""
#   def remove_articles(text):
#     regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
#     return re.sub(regex, ' ', text)
#   def white_space_fix(text):
#     return ' '.join(text.split())
#   def remove_punc(text):
#     return re.sub("\W"," ",text)
#   return white_space_fix(remove_articles(remove_punc(s))).lower().strip()

In [34]:
def white_space_fix(text):
    return ' '.join(text.split())

In [35]:
def normalize(s):
  def lower(text):
    return text.lower()
  
  def norm_apostrophe(text):
    return re.sub("’(\s|)","'",text)

  def replace_contraction(text):
    for k,v in contraction_mapping.items():
      text=text.replace(k,v)
    return text
  def remove_punc(text):
    return re.sub("\W"," ",text)

  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)

  return white_space_fix(remove_articles(remove_punc(replace_contraction((norm_apostrophe(lower(s))))))).strip()


In [36]:
train['text']=train['text'].map(normalize)

In [52]:
def find_top_common_tokens(list_texts,top_n=30):
  tokens=list(chain.from_iterable([i.split() for i in list_texts]))
  counter=Counter(tokens)
  top_common_tokens=dict(counter.most_common(top_n))
  return top_common_tokens

In [53]:
top_common_tokens=find_top_common_tokens(train['text'].tolist())

In [54]:
def remove_top_common_tokens(text):
  for k in top_common_tokens:
    text=text.replace(k," ")
  return white_space_fix(text).strip()

In [55]:
train['text']=train['text'].map(remove_top_common_tokens)

In [56]:
valid['text']=valid['text'].map(normalize)

In [57]:
# top_common_tokens=find_top_common_tokens(valid['text'].tolist())

In [58]:
# valid['text']=valid['text'].map(remove_top_common_tokens)

In [59]:
train=train[train['text']!=""]
valid=valid[valid['text']!=""]

In [60]:
len(train),len(valid)

(18094, 2589)

In [63]:
train.to_csv(join(model_path,"train.csv"),index=False)
valid.to_csv(join(model_path,"valid.csv"),index=False)

##Preprocessing

In [5]:
train=pd.read_csv(join(model_path,"train.csv"),index_col=False)
valid=pd.read_csv(join(model_path,"valid.csv"),index_col=False)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(name)

In [7]:
max_length=512
return_tensors='tf'

In [8]:
x_train=tokenizer(train['text'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)

In [9]:
y_train=tf.convert_to_tensor(train['label'],dtype=tf.int32)

In [10]:
train_dataset={i:x_train[i] for i in x_train}
train_dataset.update({"labels":y_train})

In [11]:
train_dataset=tf.data.Dataset.from_tensor_slices(train_dataset)

In [12]:
train_batch_size=8
num_training_samples=len(train)
num_training_batches=int((num_training_samples/train_batch_size)+0.5)
train_dataset=train_dataset.batch(train_batch_size, drop_remainder=True)

Valid set

In [13]:
x_valid=tokenizer(valid['text'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)

In [14]:
y_valid=tf.convert_to_tensor(valid['label'],dtype=tf.int32)

In [15]:
valid_dataset={i:x_valid[i] for i in x_valid}
valid_dataset.update({"labels":y_valid})
valid_dataset=tf.data.Dataset.from_tensor_slices(valid_dataset)

In [16]:
valid_batch_size=8
num_valid_samples=len(valid)
num_valid_batches=int((num_valid_samples/valid_batch_size)+0.5)
valid_dataset=valid_dataset.batch(valid_batch_size, drop_remainder=True)

# Train

Load the pretrained model

In [58]:
num_labels=len(train['label'].unique())
config = AutoConfig.from_pretrained(name)
config.num_labels=num_labels
model = TFAutoModelForSequenceClassification.from_pretrained(name,config=config)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
epochs=5
num_train_steps =epochs*len(train_dataset)
num_warmup_steps=600
lr=5e-5


In [60]:
loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction=tf.keras.losses.Reduction.AUTO)

In [61]:
optimizer,_=create_optimizer(init_lr=lr,num_train_steps =num_train_steps ,num_warmup_steps=num_warmup_steps,adam_epsilon =1e-8)

Define train_step function for training

In [62]:
train_step_signature= [
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, ), dtype=tf.int32),
]


@tf.function(input_signature=train_step_signature)
def train_step(input_ids, attention_mask,labels):
  

  with tf.GradientTape() as tape:

    outputs= model(input_ids=input_ids,attention_mask=attention_mask,training = True)
    loss=loss_fn(labels,outputs.logits)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss

Defind test_step function for validation

In [63]:
def test_step(input_ids, attention_mask,labels):
  
  outputs= model(input_ids=input_ids,attention_mask=attention_mask,training = False)
  loss=loss_fn(labels,outputs.logits)

  return loss

In [64]:
ckpt_path = model_path
max_to_keep=5

ckpt = tf.train.Checkpoint(model=model,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, ckpt_path, max_to_keep=max_to_keep)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print(f'Loading the latest checkpoint from {ckpt_manager.latest_checkpoint}')
else:
  print('Initializing the new checkpoint')

Initializing the new checkpoint


In [65]:
threshold=1e-5

Train

In [66]:
def run_step(dataset,session='train'):
  
  step=train_step if session=='train' else test_step

  loop = tqdm(dataset)
  loss=0

  num_batches=0
  for batch in dataset:

    input_ids=batch['input_ids']
    attention_mask=batch['attention_mask']
    labels=batch['labels']

    _loss=step(input_ids=input_ids,attention_mask=attention_mask,labels=labels)


    _loss=_loss.numpy()
    loss+=_loss
    num_batches+=1
    loop.set_postfix({f'{session}_loss_per_batch':_loss})
    loop.update()


  loss=loss/num_batches
  return loss

In [67]:
min_loss=np.inf
for epoch in range(epochs):

  """TRAINING"""
  train_loss=run_step(train_dataset)

  """Validation"""
  valid_loss=run_step(valid_dataset,session='valid')
  
  print(f'epoch: {epoch} train_loss: {train_loss} valid_loss: {valid_loss}')
  
  # if tf.abs(min_loss-valid_loss)>threshold:
  ckpt_save_path = ckpt_manager.save()
  print(f'Saving checkpoint for epoch {epoch} at {ckpt_save_path}')
    # min_loss=valid_loss

  0%|          | 0/2261 [00:00<?, ?it/s]

  0%|          | 0/323 [00:00<?, ?it/s]

epoch: 0 train_loss: 0.3373636226169765 valid_loss: 0.256614144608932
Saving checkpoint for epoch 0 at /content/drive/MyDrive/fake_news/models/bert-base-uncased/ckpt-1


  0%|          | 0/2261 [00:00<?, ?it/s]

  0%|          | 0/323 [00:00<?, ?it/s]

...

  0%|          | 0/323 [00:00<?, ?it/s]

epoch: 3 train_loss: 0.12353451258252873 valid_loss: 0.25661341326409265
Saving checkpoint for epoch 3 at /content/drive/MyDrive/fake_news/models/bert-base-uncased/ckpt-4


  0%|          | 0/2261 [00:00<?, ?it/s]

  0%|          | 0/323 [00:00<?, ?it/s]

epoch: 4 train_loss: 0.12426241976147198 valid_loss: 0.25661341326409265
Saving checkpoint for epoch 4 at /content/drive/MyDrive/fake_news/models/bert-base-uncased/ckpt-5


# Inferencing

Load test set

In [77]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [78]:
test=pd.read_csv("/content/test.csv")

In [79]:
submit=pd.read_csv("/content/submit.csv")
labels=tf.convert_to_tensor(submit['label'],dtype=tf.float32)

In [80]:
def normalize(s):
  def lower(text):
    return text.lower()
  
  def norm_apostrophe(text):
    return re.sub("’(\s|)","'",text)

  def replace_contraction(text):
    for k,v in contraction_mapping.items():
      text=text.replace(k,v)
    return text
  def remove_punc(text):
    return re.sub("\W"," ",text)

  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)

  return white_space_fix(remove_articles(remove_punc(replace_contraction((norm_apostrophe(lower(s))))))).strip()


In [81]:
def white_space_fix(text):
    return ' '.join(text.split())

In [82]:
test['text']=test['text'].astype(str).map(normalize)

In [83]:
samples=test['text'].tolist()

In [84]:
num_labels=len(submit['label'].unique())
num_labels

2

Load model

In [85]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(name)
config.num_labels=num_labels

In [86]:
model=TFAutoModelForSequenceClassification.from_pretrained(name,config=config)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [87]:
model.num_labels

2

In [88]:
max_length=512
return_tensors='tf'
tokenizer = AutoTokenizer.from_pretrained(name)

Load the weights

In [89]:
ckpt_path = model_path
max_to_keep=5

ckpt = tf.train.Checkpoint(model=model)

ckpt_manager = tf.train.CheckpointManager(ckpt, ckpt_path, max_to_keep=max_to_keep)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print(f'Loading the latest checkpoint from {ckpt_manager.latest_checkpoint}')
else:
  print('Initializing the new checkpoint')

Loading the latest checkpoint from /content/drive/MyDrive/fake_news/models/bert-base-uncased/ckpt-5


In [90]:
y_pred=[]
loop = tqdm(range(len(samples)))
for idx,sample in enumerate(samples):
  inputs=tokenizer(sample,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)
  outputs=model(**inputs,training=False)
  prob=tf.nn.softmax(outputs.logits,axis=1)
  preds=tf.argmax(prob,axis=1)
  y_pred.append(preds)
  loop.update()


  0%|          | 0/5200 [00:00<?, ?it/s]

In [91]:
y_pred=tf.convert_to_tensor(y_pred)

In [92]:
y_pred=tf.reshape(y_pred,shape=(y_pred.shape[0],))

In [93]:
y_pred=tf.cast(y_pred,dtype=tf.float32)

In [101]:
classification_report(labels,y_pred,digits=5).split("\n")

['              precision    recall  f1-score   support',
 '',
 '         0.0    0.60516   0.69132   0.64538      2339',
 '         1.0    0.71440   0.63125   0.67025      2861',
 '',
 '    accuracy                        0.65827      5200',
 '   macro avg    0.65978   0.66128   0.65782      5200',
 'weighted avg    0.66526   0.65827   0.65907      5200',
 '']

In [103]:
confusion_matrix(labels,y_pred)

array([[1617,  722],
       [1055, 1806]])